In [1]:
import pandas as pd
import numpy as np
from pycaret.datasets import get_data
from pycaret.classification import *
import matplotlib.pyplot as plt
import os

In [2]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

# Define the path to the folder containing the CSV files
file_path = os.path.join(os.getcwd(), '..', 'Datasets', 'CSE-CIC-IDS2018', 'raw', '0_01-partial-dataset.csv')
# Import csv to pandas
dataset = pd.read_csv(file_path)



dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

dataset['Timestamp'] = pd.to_datetime(dataset['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

In [3]:
# setup_pre_processing_A = setup(
#     dataset, 
#     target = 'Label',
#     date_features = ['Timestamp'],
#     create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
#     imputation_type = 'iterative',
#     numeric_iterative_imputer = 'lightgbm',
#     fix_imbalance = True,
#     fix_imbalance_method = 'SMOTE',
#     transformation = True,
#     transformation_method = 'yeo-johnson',
#     fold_strategy = 'kfold',
#     fold = 10
#     )

setup_pre_processing_B = setup(
    dataset, 
    target = 'Label',
    date_features = ['Timestamp'],
    create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
    imputation_type = 'iterative',
    numeric_iterative_imputer = 'lightgbm',
    fix_imbalance = True,
    fix_imbalance_method = 'SMOTE',
    transformation = True,
    transformation_method = 'quantile',
    fold_strategy = 'stratifiedkfold',
    fold = 10
    )

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12911
[LightGBM] [Info] Number of data points in the train set: 21932, number of used features: 70
[LightGBM] [Info] Start training from score 298398.036629
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014395 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12911
[LightGBM] [Info] Number of data points in the train set: 21932, number of used features: 70
[LightGBM] [Info] Start training from score 27704.662440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003265 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory i

,Description,Value
0,Session id,6390
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [4]:
from sklearn.metrics import precision_recall_curve, auc, fbeta_score, confusion_matrix

def pr_auc(y_true, y_pred, **kwargs):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    return auc(recall, precision)

def fbeta(y_true, y_pred, beta=1, **kwargs):
    return fbeta_score(y_true, y_pred, beta=beta)


def fpr(y_true, y_pred, **kwargs):
    # Calcula a matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    # Calcula o FPR
    return fp / (fp + tn)

In [5]:
# setup_pre_processing_A.add_metric('f2', 'F2', fbeta, greater_is_better=True, beta=2)
# setup_pre_processing_A.add_metric('pr_auc', 'PR-AUC', pr_auc, greater_is_better=True)
# setup_pre_processing_A.add_metric('fpr', 'FPR', fpr, greater_is_better=False)


setup_pre_processing_B.add_metric('f2', 'F2', fbeta, greater_is_better=True, beta=2)
setup_pre_processing_B.add_metric('pr_auc', 'PR-AUC', pr_auc, greater_is_better=True)
setup_pre_processing_B.add_metric('fpr', 'FPR', fpr, greater_is_better=False)

Name                                                               FPR
Display Name                                                       FPR
Score Function       <pycaret.internal.metrics.EncodedDecodedLabels...
Scorer               make_scorer(fpr, greater_is_better=False, resp...
Target                                                            pred
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: fpr, dtype: object

In [6]:
setup_pre_processing_B.compare_models()
df_models_comparison_B = pull().copy()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,F2,PR-AUC,FPR,TT (Sec)
ada,Ada Boost Classifier,0.9999,0.9933,0.9167,0.9750,0.9324,0.9323,0.9389,0.9324,0.9459,0.0000,13.8890
et,Extra Trees Classifier,0.9999,0.9831,0.8833,1.0000,0.9167,0.9166,0.9284,0.9167,0.9417,0.0000,7.1300
xgboost,Extreme Gradient Boosting,0.9999,0.9858,0.8667,1.0000,0.9133,0.9133,0.9230,0.9133,0.9334,0.0000,6.5400
rf,Random Forest Classifier,0.9998,0.9830,0.7833,1.0000,0.8500,0.8499,0.8698,0.8500,0.8918,0.0000,8.4040
gbc,Gradient Boosting Classifier,0.9998,0.9933,0.9167,0.9417,0.9124,0.9123,0.9205,0.9124,0.9292,0.0001,34.0610
dt,Decision Tree Classifier,0.9996,0.8749,0.7500,0.9000,0.7914,0.7912,0.8076,0.7914,0.8251,0.0001,6.7520
lightgbm,Light Gradient Boosting Machine,0.9996,0.9461,0.8333,0.8417,0.8224,0.8222,0.8296,0.8224,0.8376,0.0002,6.3650
knn,K Neighbors Classifier,0.9993,0.9164,0.7667,0.7083,0.7129,0.7125,0.7246,0.7129,0.7376,0.0004,6.6610
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5005,0.0000,5.3970
lr,Logistic Regression,0.9981,0.9891,0.8333,0.3687,0.4988,0.4981,0.5454,0.4988,0.6011,0.0017,8.9970


In [8]:
df = df_models_comparison_B.copy()

df['Score'] = df.apply(lambda row: 0.1*row['Recall'] + 0.1*row['Prec.'] + 0.2*(1-row['FPR']) + 0.3*row['F2'] + 0.3*row['PR-AUC'], axis=1)


In [10]:
df = df[['Prec.','Recall', 'FPR', 'PR-AUC', 'F2', 'Score']]

In [11]:
df

,Prec.,Recall,FPR,PR-AUC,F2,Score
ada,0.9750,0.9167,0.0000,0.9459,0.9324,0.95266
et,1.0000,0.8833,0.0000,0.9417,0.9167,0.94585
xgboost,1.0000,0.8667,0.0000,0.9334,0.9133,0.94068
rf,1.0000,0.7833,0.0000,0.8918,0.8500,0.90087
gbc,0.9417,0.9167,0.0001,0.9292,0.9124,0.93830
dt,0.9000,0.7500,0.0001,0.8251,0.7914,0.84993
lightgbm,0.8417,0.8333,0.0002,0.8376,0.8224,0.86546
knn,0.7083,0.7667,0.0004,0.7376,0.7129,0.78257
dummy,0.0000,0.0000,0.0000,0.5005,0.0000,0.35015
lr,0.3687,0.8333,0.0017,0.6011,0.4988,0.64983


In [36]:
def compute_score(fbeta, pr_auc, fpr, recall, precision):
    return 0.3*fbeta + 0.3*pr_auc + 0.2*(1-fpr) + 0.1*recall + 0.1*precision


def evaluate_algorithms(df):
    df.rename(columns={'Prec.': 'Precision'}, inplace=True)
    df['Score'] = df.apply(
        lambda row: compute_score(
            row['F5'],
            row['PR-AUC'],
            row['FPR'],
            row['Recall'],
            row['Precision']
        ),
        axis = 1 
    )

    df = df[['Model', 'Precision', 'Recall', 'FPR', 'PR-AUC', 'F5', 'Score']]

    return df.copy()

In [37]:
df_models_comparison_B

,Model,Accuracy,AUC,Recall,Precision,F1,Kappa,MCC,F5,PR-AUC,FPR,TT (Sec),Score
ada,Ada Boost Classifier,0.9999,1.0000,0.9667,0.9667,0.9667,0.9666,0.9666,0.9667,0.9667,0.0000,13.391,0.97336
gbc,Gradient Boosting Classifier,0.9999,1.0000,1.0000,0.9267,0.9550,0.9549,0.9591,0.9550,0.9633,0.0001,34.831,0.96814
rf,Random Forest Classifier,0.9998,1.0000,0.8000,0.9000,0.8333,0.8333,0.8414,0.8333,0.9001,0.0000,8.999,0.89002
xgboost,Extreme Gradient Boosting,0.9998,1.0000,0.8667,0.9417,0.8790,0.8789,0.8912,0.8790,0.9042,0.0001,6.615,0.91578
et,Extra Trees Classifier,0.9997,0.9979,0.8000,0.9333,0.8300,0.8299,0.8474,0.8300,0.8668,0.0001,6.172,0.88235
lightgbm,Light Gradient Boosting Machine,0.9997,0.9633,0.7667,0.8500,0.7848,0.7847,0.7962,0.7848,0.8584,0.0001,6.125,0.85461
dt,Decision Tree Classifier,0.9995,0.8749,0.7500,0.8583,0.7757,0.7755,0.7894,0.7757,0.8043,0.0002,5.977,0.83479
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5005,0.0000,5.409,0.35015
knn,K Neighbors Classifier,0.9979,0.8326,0.5667,0.3262,0.3619,0.3610,0.4004,0.3619,0.4467,0.0017,6.482,0.53153
lr,Logistic Regression,0.9875,0.9938,0.8333,0.0732,0.1320,0.1303,0.2388,0.1320,0.4533,0.0123,6.907,0.46378


In [38]:
df_results_B = evaluate_algorithms(df_models_comparison_B)
df_results_B = df_results_B.sort_values(by=['Score'], ascending=False)
df_results_B

,Model,Precision,Recall,FPR,PR-AUC,F5,Score
ada,Ada Boost Classifier,0.9667,0.9667,0.0000,0.9667,0.9667,0.97336
gbc,Gradient Boosting Classifier,0.9267,1.0000,0.0001,0.9633,0.9550,0.96814
xgboost,Extreme Gradient Boosting,0.9417,0.8667,0.0001,0.9042,0.8790,0.91578
rf,Random Forest Classifier,0.9000,0.8000,0.0000,0.9001,0.8333,0.89002
et,Extra Trees Classifier,0.9333,0.8000,0.0001,0.8668,0.8300,0.88235
lightgbm,Light Gradient Boosting Machine,0.8500,0.7667,0.0001,0.8584,0.7848,0.85461
dt,Decision Tree Classifier,0.8583,0.7500,0.0002,0.8043,0.7757,0.83479
knn,K Neighbors Classifier,0.3262,0.5667,0.0017,0.4467,0.3619,0.53153
lr,Logistic Regression,0.0732,0.8333,0.0123,0.4533,0.1320,0.46378
lda,Linear Discriminant Analysis,0.0169,0.9667,0.0616,0.4918,0.0331,0.44351


In [39]:
df_final = df_results_B.copy()

In [40]:
df_final

,Model,Precision,Recall,FPR,PR-AUC,F5,Score
ada,Ada Boost Classifier,0.9667,0.9667,0.0000,0.9667,0.9667,0.97336
gbc,Gradient Boosting Classifier,0.9267,1.0000,0.0001,0.9633,0.9550,0.96814
xgboost,Extreme Gradient Boosting,0.9417,0.8667,0.0001,0.9042,0.8790,0.91578
rf,Random Forest Classifier,0.9000,0.8000,0.0000,0.9001,0.8333,0.89002
et,Extra Trees Classifier,0.9333,0.8000,0.0001,0.8668,0.8300,0.88235
lightgbm,Light Gradient Boosting Machine,0.8500,0.7667,0.0001,0.8584,0.7848,0.85461
dt,Decision Tree Classifier,0.8583,0.7500,0.0002,0.8043,0.7757,0.83479
knn,K Neighbors Classifier,0.3262,0.5667,0.0017,0.4467,0.3619,0.53153
lr,Logistic Regression,0.0732,0.8333,0.0123,0.4533,0.1320,0.46378
lda,Linear Discriminant Analysis,0.0169,0.9667,0.0616,0.4918,0.0331,0.44351


In [41]:
# Define the path to save the dataset pre-processed
file_path = os.path.join(os.getcwd(), '..', 'Scripts', 'models_comparison', 'pycaret-models-comparison.csv')

# Save the dataset pre-processed
df_final.to_csv(file_path ,index=False)